In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from sklearn.model_selection import KFold

def cross_validation_split(URM_all, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    for train_indices, test_indices in kf.split(URM_all):
        URM_train = URM_all[train_indices]
        URM_test = URM_all[test_indices]
        yield URM_train, URM_test


In [3]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, \
    MultiThreadSLIM_SLIMElasticNetRecommender

model = MultiThreadSLIM_SLIMElasticNetRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2627 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [4]:
import optuna as op
import numpy as np

def objective(trial):
    # Hyperparameters to optimize
    topK = trial.suggest_int("topK", 3, 1e4)
    l1 = trial.suggest_float("l1_ratio", 0.01e-05, 1, log=True)
    alpha = trial.suggest_float("alpha", 5e-4, 1)

    MAP_scores = []

    for URM_train, URM_validation in cross_validation_split(URM_all, n_folds=5):
        # Model initialization and fitting
        recommender = model(URM_train)
        recommender.fit(topK=topK, l1_ratio=l1, alpha=alpha, workers=7)

        # Evaluation
        evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
        result, _ = evaluator_validation.evaluateRecommender(recommender)
        MAP_result = result["MAP_MIN_DEN"].item()
        MAP_scores.append(MAP_result)

    # Return the average MAP across all folds
    return np.mean(MAP_scores)

In [ ]:
best_params = {
    "topK": 8894,
    "l1_ratio": 0.05565733019999427,
    "alpha": 0.0012979360257937668
}
study_name = "slim-elastic-crossvalidation-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=5)

In [ ]:
study.best_params

In [ ]:
final = model(URM_train_validation)
final.fit(**study.best_params, workers=7)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

Original scores

In [ ]:
## MAP_MIN_DEN = .0822900 ## MAP = 0.0483066 the best 0.139

In [ ]:
## MAP_MIN_DEN = .0845288 ## MAP = 0.0495724 second 0.135

In [ ]:
## MAP_MIN_DEN = .0846506 ## MAP = 0.0496060 third 